In [15]:
import os
import sys
import pickle
import pandas as pd
import tensorflow as tf

sys.path.append(os.path.abspath("../"))

In [14]:
df_kuhp = pd.read_csv("../data/dataset_csv/dataset_kuhp.csv")

In [48]:
df_kuhp["SUB_KLASIFIKASI"] = df_kuhp["SUB_KLASIFIKASI"].map(lambda text: text.replace(" ", "-"))

df_kuhp.head()

,BUKU,BAB,PASAL,ISI_PASAL,SUB_KLASIFIKASI
0,NaN,BAB I,Pasal 1,"(1) Suatu perbuatan tidak dapat dipidana, kec...",tidak-diketahui
1,NaN,BAB I,Pasal 2,Ketentuan pidana dalam perundang-undang an dan...,tidak-diketahui
2,NaN,BAB I,Pasal 3,Ketentuan pidana dalam perundang-undangan Indo...,pelanggaran-lalu-lintas
3,NaN,BAB I,Pasal 4,Ketentuan pidana dalam perundang-undang an Ind...,pemalsuan
4,NaN,BAB I,Pasal 5,(1) Ketentuan pidana dalam perundang-undangan...,tidak-diketahui


In [5]:
df_pidana_umum = pd.read_csv("../data/dataset_csv/dataset_pidana_umum.csv")

## Model 1

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

model = tf.keras.models.load_model("../models/model_klasifikasi_pidana.h5", compile=False)

# open file tokenizer.pkl
with open('../models/tokenizer.pkl', 'rb') as file:
   tokenizer = pickle.load(file)

In [12]:
df_pidana_umum = pd.read_csv("../data/dataset_csv/dataset_pidana_umum.csv", encoding="utf-8", usecols=["riwayat_dakwaan", "sub_klasifikasi"])
y = df_pidana_umum["sub_klasifikasi"]

sub_classification = y.unique()
class_mapping = {label: idx for idx, label in enumerate(sub_classification)}
y_encoded = y.map(class_mapping)

def predict_pasal(input_text):
   input_seq = tokenizer.texts_to_sequences([input_text])
   input_padded = pad_sequences(input_seq, maxlen=300, padding='post', truncating='post')
   prediction = model.predict(input_padded)
   predicted_class = sub_classification[prediction.argmax()]
   return predicted_class

FileNotFoundError: [Errno 2] No such file or directory: '../data/dataset_csv/dataset_pidana_umum.csv'

In [ ]:
predicted_pasal = predict_pasal("terdakwa [Nama Lengkap Terdakwa] dengan sengaja dan tanpa hak mengambil barang milik [Nama Korban], yaitu [Deskripsi Barang yang Dicuri]. Sebelumnya, terdakwa telah merencanakan aksi pencurian dengan membawa peralatan untuk membobol pintu. Setelah berhasil masuk, terdakwa mengambil barang curian dan langsung melarikan diri. Perbuatan terdakwa ini didukung oleh keterangan saksi [Nama Saksi] yang melihat terdakwa keluar dari rumah korban serta barang bukti berupa [Deskripsi Barang Bukti]. Berdasarkan fakta-fakta tersebut, Jaksa Penuntut Umum berpendapat bahwa terdakwa terbukti secara sah dan meyakinkan melakukan tindak pidana pencurian sebagaimana diatur dalam Pasal 362 KUHP dan menuntut pidana penjara selama [Lama Hukuman] tahun, denda sebesar [Jumlah Denda], dan biaya perkara.")

print(predicted_pasal)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step
penipuan


In [ ]:
pasal = df_kuhp[df_kuhp["SUB_KLASIFIKASI"] == predicted_pasal]

pasal = pasal[["PASAL", "ISI_PASAL"]]

pasal = pasal.to_dict('records')

In [ ]:
for p in pasal:
   print(p["PASAL"])
   print(p["ISI_PASAL"])

Pasal 185
Barang siapa dalam perkelahian tanding merampas nyawa pihak lawan atau melukai tubuhnya, maka diterapkan ketentuan-ketentuan mengenai pembunuhan berencana, pembunuhan atau penganiayaan: 1. jika persyaratan tidak diatur terlebih dahulu; 2. jika perkelahian tanding tidak dilakukan di hadapan saksi kedua belah pihak; 3. jika pelaku dengan sengaja dan meru gikan pihak lawan, bersalah melakukan perbuatan penipuan atau yang me nyimpang dari persyaratan.
Pasal 186
(1) Para saksi dan dokter yang menghadiri perkelahian tanding, tidak dipidana. (2) Para saksi diancam: Biro Hukum dan Humas Badan Urusan Administrasi Mahkamah Agung-RI /g3/g3  1. dengan pidana penjara paling lama tiga tahun, jika persyaratan tidak diatur terlebih dahulu, atau jika para saksi menghasut para pihak untuk perkelahian tanding; 2. dengan pidana penjara paling lama empat tahun, jika para saksi dengan sengaja dan merugikan salah satu at au kedua belah pihak, bersalah melakukan perbuatan penipuan atau membiarkan pa

## Model 2

In [3]:
from transformers import TFBertModel, AutoTokenizer

c:\Users\LENOVO\.virtualenvs\Machine_Learning-ryZF-xD4\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")

with open("../models/label_encoder.pkl", 'rb') as file:
   label_encoder = pickle.load(file)

c:\Users\LENOVO\.virtualenvs\Machine_Learning-ryZF-xD4\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
MODEL_PATH = "../models/model_2.h5"

model = tf.keras.models.load_model(MODEL_PATH, custom_objects={
    "TFBertModel": TFBertModel
})

In [ ]:
def predict_pasal(input_text):
    tokenized = tokenizer(
        input_text, max_length=128, padding="max_length", truncation=True, return_tensors="tf"
    )
   
    prediction = model.predict({
      "input_ids": tokenized['input_ids'],
      "attention_mask": tokenized['attention_mask']
   })

    predicted_pasal = label_encoder.inverse_transform(prediction.argmax(axis=1))

    return predicted_pasal[0]
    

In [10]:
example_predict = predict_pasal('terdakwa [Nama Lengkap Terdakwa] dengan sengaja dan tanpa hak mengambil barang milik [Nama Korban], yaitu [Deskripsi Barang yang Dicuri]. Sebelumnya, terdakwa telah merencanakan aksi pencurian dengan membawa peralatan untuk membobol pintu. Setelah berhasil masuk, terdakwa mengambil barang curian dan langsung melarikan diri. Perbuatan terdakwa ini didukung oleh keterangan saksi [Nama Saksi] yang melihat terdakwa keluar dari rumah korban serta barang bukti berupa [Deskripsi Barang Bukti]. Berdasarkan fakta-fakta tersebut, Jaksa Penuntut Umum berpendapat bahwa terdakwa terbukti secara sah dan meyakinkan melakukan tindak pidana pencurian, dan biaya perkara.')

1/1 [==============================] - 0s 223ms/step


In [13]:
print(example_predict)

['pencurian']
